In [92]:
import mysql.connector
import pandas as pd
from datetime import datetime, date, timedelta

def query_data(host:str, user:str, password, database_name):
    try:
        with mysql.connector.connect(
            host= host,
            user = user,
            password = password,
            database = database_name
        )as con:
            if con.is_connected():
                print("connection to DIT_EPD successfully, \nWELCOME TO EPD DATABASE!")

                with con.cursor() as cur:
                    sql_cmd =  """
                    SELECT date_info.case_id, date_info.sent_date, supplier_info.supplier_name, supplier_info.lt_normal
                    FROM date_info
                    JOIN main ON main.case_id = date_info.case_id
                    JOIN supplier_info ON main.supplier_id = supplier_info.id
                    where main.req_supplier = true and sent_date is not null;
                    """
                    cur.execute(sql_cmd)
                    rows = cur.fetchall()


                    df = pd.DataFrame(rows, columns=['id', 'sent_date', 'supplier_name', 'lt'])
                    return df







    except Exception as e:
        print(f'error => {e}')

def leadtime_calculate(x):
    sent_date = x['sent_date']
    leadtime = x['lt']
    count = 0
    due_date =sent_date
    while count < leadtime:
        due_date = due_date + timedelta(days=1)
        if due_date.isoweekday() not in (6,7):
            count = count + 1
    return  due_date

def determine_leadtime(x, reminder_day):
    due_date = x['due_date']
    if due_date < date.today() + timedelta(days=reminder_day):
        status = 'overdue'
    else:
        status = 'on track'
    return status


if __name__ == '__main__':
    df= query_data('localhost', 'root', 'ditepd', 'pcb')
    df['due_date'] = df.apply(leadtime_calculate, axis=1)
    df['status'] = df.apply(determine_leadtime, axis=1, reminder_day = 3)
    print(df)

connection to DIT_EPD successfully, 
WELCOME TO EPD DATABASE!
     id   sent_date  supplier_name  lt    due_date    status
0    24  2024-05-27         KATATA  30  2024-07-08  on track
1    74  2024-05-27         REXXAM  11  2024-06-11   overdue
2    81  2024-05-27         REXXAM  11  2024-06-11   overdue
3    82  2024-05-27         REXXAM  11  2024-06-11   overdue
4    85  2024-05-27         REXXAM  11  2024-06-11   overdue
5    89  2024-05-27         REXXAM  11  2024-06-11   overdue
6    90  2024-05-27         REXXAM  11  2024-06-11   overdue
7    94  2024-05-27         REXXAM  11  2024-06-11   overdue
8   131  2024-05-27            SET  45  2024-07-29  on track
9   132  2024-05-27            SET  45  2024-07-29  on track
10  135  2024-05-27            SET  45  2024-07-29  on track
11  137  2024-05-27            SET  45  2024-07-29  on track
12  142  2024-06-10         KATATA  30  2024-07-22  on track
13  143  2024-06-10         KATATA  30  2024-07-22  on track
14  158  2024-06-10  KA

In [93]:
df1 = df.set_index(['supplier_name','status'])

In [94]:
df1

id   sent_date  lt    due_date
supplier_name status                                   
KATATA        on track   24  2024-05-27  30  2024-07-08
REXXAM        overdue    74  2024-05-27  11  2024-06-11
              overdue    81  2024-05-27  11  2024-06-11
              overdue    82  2024-05-27  11  2024-06-11
              overdue    85  2024-05-27  11  2024-06-11
              overdue    89  2024-05-27  11  2024-06-11
              overdue    90  2024-05-27  11  2024-06-11
              overdue    94  2024-05-27  11  2024-06-11
SET           on track  131  2024-05-27  45  2024-07-29
              on track  132  2024-05-27  45  2024-07-29
              on track  135  2024-05-27  45  2024-07-29
              on track  137  2024-05-27  45  2024-07-29
KATATA        on track  142  2024-06-10  30  2024-07-22
              on track  143  2024-06-10  30  2024-07-22
KAGA BRANCH 1 on track  158  2024-06-10  17  2024-07-03
REXXAM        overdue   166  2024-06-10  11  2024-06-25
              overdue   167  2024-06-10  11  2024-06-25
              on track  168  2024-06-17  11  2024-07-02

In [95]:
import mysql.connector
import pandas as pd
from datetime import datetime, date, timedelta

def query_data(host:str, user:str, password, database_name):
    try:
        with mysql.connector.connect(
            host= host,
            user = user,
            password = password,
            database = database_name
        )as con:
            if con.is_connected():
                print("connection to DIT_EPD successfully, \nWELCOME TO EPD DATABASE!")

                with con.cursor() as cur:
                    sql_cmd =  """
                    select * from customer;
                    """
                    cur.execute(sql_cmd)
                    rows = cur.fetchall()


                    df = pd.DataFrame(rows, columns=['id', 'fname', 'lname'])
                    return df

    except Exception as e:
        print(f'error => {e}')


if __name__ == '__main__':
    df= query_data('localhost', 'root', 'ditepd', 'test_cleaning_python')
    print(df)

connection to DIT_EPD successfully, 
WELCOME TO EPD DATABASE!
   id  fname   lname
0   1   John     Doe
1   2  peTer  pARKER
2   3   Adam   sMITH


In [96]:
df.set_index('id', inplace=True)

In [97]:
df['fname_edited'] = df.apply(lambda x: x['fname'].title(), axis=1)


In [98]:
conn = mysql.connector.connect(host='localhost', user='root', password='ditepd', database='test_cleaning_python')
for i in df.index:
    id = i
    name = df.loc[i,'fname_edited']
    sql_cmd_update = """
    update customer set fname = %s where id = %s    
    """
    cursor = conn.cursor()
    cursor.execute(sql_cmd_update, (name, id))
    conn.commit()
    cursor.close()
conn.close()

    
